데이터 불러오기

In [5]:
import pandas as pd
PATH = 'F:/Projects/topic-modeling/datasets/everytime.csv'

df = pd.read_csv(PATH, sep='\t')['content']
df

0       안녕하세요 MBC PD수첩 팀입니다 저희는 대학생들의 주거실태를 다루는 방송을 제작...
1                                               개강이라니 개같네
2       my mental을 아작내고있어.....☆ 전공.....없다....듣고싶은....n...
3       고려대학교 생태연구회 열두루달에서 신입 부원을 모집합니다! 열두루달은 일 년 열두 ...
4                                            통학러는 웁니다 ㅠ ㅠ
                              ...                        
5820    미루고 미루던 발송일 당일부터 이틀 째 연락 두절  영수증이고 뭐고 제시한 적 없음...
5821    가고싶은 공기업 체험형인턴 합격했다...! ㅠㅠ 누군가에게는 별거 아닌거지만 (실제...
5822                                     'Z축을' 박차고 나오기 때문
5823    원래는 고딩때 일진 눈도 못마주치고 쓰레기  아무대나 버리고 침 찍찍 뱉고 했는데 ...
5824         그러면 성대가 탁월하시다는 건데 왜 성대를 안 가고 고대에 왔을까요? 궁금합니다
Name: content, Length: 5825, dtype: object

KKma, Komoran, Okt비교

In [23]:
from konlpy.tag import Kkma
kkma = Kkma()

text_list = df.tolist()

tokenized_text = []
for text in text_list[:1000]:
    try:
        preprocessed = kkma.nouns(text)
        tokenized_text.append(preprocessed)  
    except:
        continue

명사 추출이 이상할 경우 직접 명사를 추가하자
https://now0930.pe.kr/wordpress/?p=3134

dictionary만들기

In [25]:
from gensim import corpora
id2word = corpora.Dictionary(tokenized_text)
id2word.filter_extremes(no_below = 20) #20회 이하로 등장한 단어는 삭제

corpus = [id2word.doc2bow(i) for i in tokenized_text]

In [26]:
import gensim
NUM_TOPICS = 10

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=id2word, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.184*"말" + 0.116*"때문" + 0.114*"수" + 0.101*"라"')
(1, '0.171*"글" + 0.133*"개" + 0.091*"월" + 0.088*"에타"')
(2, '0.119*"나" + 0.110*"학교" + 0.090*"고대" + 0.071*"거"')
(3, '0.097*"전" + 0.094*"우리" + 0.088*"일" + 0.067*"년"')
(4, '0.196*"거" + 0.154*"사람" + 0.081*"뭐" + 0.055*"자기"')
(5, '0.142*"때" + 0.089*"중" + 0.062*"미" + 0.061*"문제"')
(6, '0.118*"1" + 0.117*"2" + 0.072*"명" + 0.062*"대"')
(7, '0.184*"분" + 0.123*"이번" + 0.079*"학점" + 0.077*"어디"')
(8, '0.126*"공부" + 0.084*"거" + 0.080*"ㅇ" + 0.077*"생각"')
(9, '0.167*"오늘" + 0.117*"시험" + 0.079*"저" + 0.076*"학우"')


시각화해보기

In [27]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(ldamodel, corpus, id2word)
pyLDAvis.display(vis)

문서 별 토빅 분포 찾기

In [28]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(0, 0.016668186), (1, 0.016670993), (2, 0.8499767), (3, 0.016666759), (4, 0.016666759), (5, 0.016675517), (6, 0.01667061), (7, 0.016668512), (8, 0.016669147), (9, 0.016666759)]
1 번째 문서의 topic 비율은 [(0, 0.050000053), (1, 0.54998964), (2, 0.050001945), (3, 0.050000053), (4, 0.050000053), (5, 0.050000053), (6, 0.05000487), (7, 0.050000053), (8, 0.050003234), (9, 0.050000053)]
2 번째 문서의 topic 비율은 [(0, 0.05000007), (1, 0.05000007), (2, 0.05000007), (3, 0.5499872), (4, 0.050005462), (5, 0.05000007), (6, 0.050000075), (7, 0.050004628), (8, 0.05000007), (9, 0.050002288)]
3 번째 문서의 topic 비율은 [(0, 0.011111871), (1, 0.011112339), (2, 0.40610963), (3, 0.3577695), (4, 0.01111153), (5, 0.01111291), (6, 0.011112197), (7, 0.011112953), (8, 0.1583304), (9, 0.011116705)]
4 번째 문서의 topic 비율은 [(0, 0.1), (1, 0.1), (2, 0.1), (3, 0.1), (4, 0.1), (5, 0.1), (6, 0.1), (7, 0.1), (8, 0.1), (9, 0.1)]


In [29]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [30]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

AttributeError: 'DataFrame' object has no attribute 'append'